# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **LAB 05: Transformations and Actions** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez
**Alumnos**: Angel Ramirez Carrillo, Samuel Alejandro Romero Mora, Yochabel Martinez Cazares, Roberto Osorno

In [1]:
%pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [2]:
import findspark
from pyspark.sql.functions import col, sum, avg
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Transformations-Actions") \
    .master("spark://5b6f136ea5a4:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

### Transformations

In [5]:
import importlib
import whatsapp2.spark_utils

importlib.reload(whatsapp2.spark_utils)

columns_info = [ ("track_id", "string"),
                ("track_name", "string"),
                ("artist_id", "string"),
                ("artist_name", "string"),
                ("album_id", "string"),
                ("duration", "integer"),
                ("release_date", "timestamp"),
                ("popularity", "integer"),
                ("danceability", "double"),
                ("energy", "double"),
                ("key", "integer"),
                ("loudness", "double"),
                ("mode", "integer"),
                ("speechiness", "double"),
                ("acousticness", "double"),
                ("instrumentalness", "double"),
                ("liveness", "double"),
                ("valence", "double"),
                ("tempo", "double"),
                ("playlist_id", "string"),
                ("playlist_name", "string"),
                ("duration_mins", "double"),
                ("genre", "string")]

schema = whatsapp2.spark_utils.SparkUtils.generate_schema(columns_info)

# Create DataFrame
tiktok_df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/tiktok.csv")

print(schema)

StructType([StructField('track_id', StringType(), True), StructField('track_name', StringType(), True), StructField('artist_id', StringType(), True), StructField('artist_name', StringType(), True), StructField('album_id', StringType(), True), StructField('duration', IntegerType(), True), StructField('release_date', TimestampType(), True), StructField('popularity', IntegerType(), True), StructField('danceability', DoubleType(), True), StructField('energy', DoubleType(), True), StructField('key', IntegerType(), True), StructField('loudness', DoubleType(), True), StructField('mode', IntegerType(), True), StructField('speechiness', DoubleType(), True), StructField('acousticness', DoubleType(), True), StructField('instrumentalness', DoubleType(), True), StructField('liveness', DoubleType(), True), StructField('valence', DoubleType(), True), StructField('tempo', DoubleType(), True), StructField('playlist_id', StringType(), True), StructField('playlist_name', StringType(), True), StructField(

#### Collect all rows as a list to the driver program

In [12]:
rows = tiktok_df.collect()
rows

[Row(track_id='6kVuF2PYLuvl9T85XjNbaO', track_name='Lay It Down Gmix - Main', artist_id='1Xfmvd48oOhEWkscWyEbh9', artist_name='Lloyd', album_id='43C6GVlhXG4KfZuEbxty3r', duration=302186, release_date=datetime.datetime(2011, 1, 1, 0, 0), popularity=28, danceability=0.597, energy=0.8, key=1, loudness=-5.422999999999999, mode=0, speechiness=0.312, acousticness=0.0461, instrumentalness=0.0, liveness=0.18, valence=0.565, tempo=155.93200000000002, playlist_id='6kVuF2PYLuvl9T85XjNbaO', playlist_name='6kVuF2PYLuvl9T85XjNbaO', duration_mins=5.036433333333333, genre='TIKTOK DANCE'),
 Row(track_id='1RGIjMFMgJxkZHMDXVYzOJ', track_name='Bartender (feat. Akon)', artist_id='3aQeKQSyrW4qWr35idm0cy', artist_name='T-Pain', album_id='6CrSEKCF4TYrbSIitegb3h', duration=238800, release_date=datetime.datetime(2007, 6, 5, 0, 0), popularity=75, danceability=0.8320000000000001, energy=0.391, key=8, loudness=-8.504, mode=1, speechiness=0.0628, acousticness=0.0564, instrumentalness=0.0, liveness=0.223999999999999

In [16]:
df_select = tiktok_df.select("popularity")
df_select = tiktok_df.filter(col("popularity")>80)

count = df_select.count()

count

1023

In [17]:
df_select = tiktok_df.select("genre","duration_mins")
df_select = tiktok_df.groupBy("genre").agg(avg("duration_mins").alias("avg_duration"))
df_select.show()

+------------------+------------------+
|             genre|      avg_duration|
+------------------+------------------+
|TIKTOK PHILIPPINES|3.2801328435737513|
|      TIKTOK DANCE| 3.015020713916861|
|           _TIKTOK| 3.251196442168827|
|        TIKTOK OPM| 4.257192861885788|
+------------------+------------------+



In [21]:
df_select = tiktok_df.select("energy")
df_select =tiktok_df.orderBy(col("energy").desc()).limit(5)

df_select.show()


+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------------+----------+------------------+------------------+---+------------------+----+-----------+--------------------+-------------------+--------+------------------+------------------+--------------------+--------------------+-----------------+----------+
|            track_id|          track_name|           artist_id|         artist_name|            album_id|duration|       release_date|popularity|      danceability|            energy|key|          loudness|mode|speechiness|        acousticness|   instrumentalness|liveness|           valence|             tempo|         playlist_id|       playlist_name|    duration_mins|     genre|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------+-------------------+----------+------------------+------------------+---+------------------+----+-----------+--------

In [20]:
df_select = tiktok_df.select("playlist_id","duration")
df_select = tiktok_df.groupBy("playlist_id").agg(sum("duration").alias("total_time_play_list"))
df_select.show()

+--------------------+--------------------+
|         playlist_id|total_time_play_list|
+--------------------+--------------------+
|5IZc3KIVFhjzJ0L2k...|              448480|
|08ia51KbTcfs4QVT5...|              248910|
|7xVLFuuYdAvcTfcP3...|              567386|
|2RBILNmyq8p4fqVWO...|              129776|
|6GdDjthxbTGBV9rl2...|              199255|
|7krYEnB1OI1RbnJBa...|              125746|
|1FgPyHX7HruKDL4Tx...|              146688|
|62RtxFf9epYNWOUHJ...|              160166|
|5ow0sNF1zSqp71Ix5...|              164009|
|0LlJbV4lyzJYE14YC...|              642548|
|6NFKf8vBApSvtzkap...|              222446|
|5P8lyudWE7HQxb4lu...|              255040|
|2BgEsaKNfHUdlh97K...|              186986|
|7F9vK8hNFMml4GtHs...|              190427|
|4vVTI94F9uJ8lHNDW...|              201946|
|2uULRpRtKhCdojXwo...|              132000|
|1tRlGMHsf21FDo6pj...|              107403|
|215fAfwkWtlj30ofd...|              139285|
|3bidbhpOYeV4knp8A...|              258342|
|0YFocHKmrMme7Isel...|          

In [31]:
# Stop the SparkContext
sc.stop()